In [2]:
import requests
from click.testing import CliRunner
from pprint import pprint, pformat
import pandas

from requests.auth import HTTPBasicAuth

from kf_model_fhir.config import FHIR_VERSION, SERVER_CONFIG, PROJECT_DIR
from kf_model_fhir import cli
from kf_model_fhir.client import FhirApiClient
from kf_model_fhir.loader import load_resources
from kf_model_fhir.utils import read_json
from kf_model_fhir.aidbox import *

SERVER_CONFIG.pop('simplifier', None)
runner = CliRunner()

def emoji(resp):
    total = resp.get('total', None)
    if total is None:
        return '❌'
    if total > 0:
        return '✅'
    else:
        return '⚠️ or ✅'

def execute_queries(queries, display_content=False):
    for query in queries:
        dlen = len(query['desc'])
        print('*' * dlen)
        print(query['desc'])
        print('*' * dlen)
        for server_name, settings in SERVER_CONFIG.items():
            print(server_name.upper())
            print('-' * len(server_name))
            url = f"{settings['base_url']}/{query['endpoint']}"
            uname = settings.get('username')
            pw = settings.get('password')
            params = query['params']
            params.update({'_total': 'accurate'})
            resp, status_code = get(url, username=uname, password=pw, params=params)

            if display_content:
                pprint(resp)
                
            print(f"{emoji(resp)} Found {resp.get('total')} {query['endpoint'].lstrip('/')} "
                  f"matching these params {pformat(params)}")
            if resp.get('total') == counts[query['endpoint']]:
                print(
                    f'⚠️ ？ The returned total = actual count of {query["endpoint"]}. '
                    f'This may or may not be correct. Some servers ignore search parameters '
                    'that they do not understand, which results in a get all query. '
                )
            print('\n')

## FHIR Servers

In [ ]:
print(f'Evaluating servers: {pformat(SERVER_CONFIG.keys())}')

### Microsoft FHIR Server on Azure

Pros
- Open source
- Free
- A "straight" FHIR server that adheres to FHIR spec
- Good error handling and reporting
    - Reports when a search param or operation is not supported

Cons
- Only runs on Azure
- Very little documentation
- Does not support a lot of major search functionality
    - Doesn't support full text search
    - Doesn't support custom queries
    - Chaining/reverse chaining
    - Including references


### Vonk

Pros
- A "straight" FHIR server that adheres to FHIR spec
- Has ~70% of search API spec implemented
- Decent documentation
- Vonk team is fairly responsive to issue reports (0.5 - 1day) on Zulip

Cons
- Not open source
- Not free
- Production releases are very buggy
- Not as quick to turn around fixes and releases as Aidbox
- Ignores search parameters it doesn't understand and returns everything
- Error handling and reporting is confusing or lacking
- Has a limited policy engine for access control
- Doesn't support full text search
- Doesn't support custom queries


### HAPI (Smile CDR)

Pros
- A "straight" FHIR server that adheres to FHIR spec
- Has 70% of search API spec implemented
- Looks like professional/detailed documentation
- Looks like there are a lot of features - haven't had time to delve into them all
- Built on top of HAPI, open source FHIR server
- Good error handling and reporting
    - Reports when a search param or operation is not supported
- Supports full text search
- Supports multitenancy and seems to have a lot of features relating to auth

Cons
- Not free
- Not sure if it supports some of the basic search parameters

### Aidbox

Pros
- Has 70% of search API spec implemented
- Supports special search features that make it very flexible
    - Full text search
    - Query by SQL
    - Save SQL queries as endpoints
    - Query by exact path
- Decent documentation
- Team is very responsive to issues and makes releases often
- Good error handling and reporting
    - Reports when a search param or operation is not supported
- Aidbox representation of conformance resources is much easier to digest
  and understand than FHIR conformance resources
- Uses PostgresSQL as their database and its open source
- Has a GraphQL API - haven't tested it yet

Cons
- Represents conformance resources (e.g. extensions, search parameters in their own way
    - Until they've developed the tools to easily go between Aidbox and FHIR, we will 
      have to do this ourselves
- Does not seem to support the :missing: modifier for search

## Generate Data

In [ ]:
# Generate resources
result = runner.invoke(cli.generate,['./resources'])
assert result.exit_code == 0

## Data Description

In [5]:
print('*********************')
print('Data Description')
print('*********************')
resources = load_resources(os.path.join(PROJECT_DIR, 'resources'))
df = pandas.DataFrame(
    [
        {'resource_type': r['resource_type'],
         'id': r['content'].get('id'),
         'references': r['content'].get('subject', {}).get('reference'),
        }
        for r in resources
    ]
)
display(df)
counts = df.groupby(['resource_type']).size()
display(counts)

2019-11-13 17:50:17,197 - kf_model_fhir.loader - DEBUG - Reading resource file: /Users/singhn4/Projects/kids_first/kf-model-fhir/project/resources/Condition-CD-00000-0.json
2019-11-13 17:50:17,199 - kf_model_fhir.loader - DEBUG - Reading resource file: /Users/singhn4/Projects/kids_first/kf-model-fhir/project/resources/Condition-CD-00001-0.json
2019-11-13 17:50:17,200 - kf_model_fhir.loader - DEBUG - Reading resource file: /Users/singhn4/Projects/kids_first/kf-model-fhir/project/resources/Condition-CD-00002-0.json
2019-11-13 17:50:17,202 - kf_model_fhir.loader - DEBUG - Reading resource file: /Users/singhn4/Projects/kids_first/kf-model-fhir/project/resources/Condition-CD-00003-0.json
2019-11-13 17:50:17,203 - kf_model_fhir.loader - DEBUG - Reading resource file: /Users/singhn4/Projects/kids_first/kf-model-fhir/project/resources/Condition-CD-00004-0.json
2019-11-13 17:50:17,204 - kf_model_fhir.loader - DEBUG - Reading resource file: /Users/singhn4/Projects/kids_first/kf-model-fhir/projec

2019-11-13 17:50:17,297 - kf_model_fhir.loader - DEBUG - Reading resource file: /Users/singhn4/Projects/kids_first/kf-model-fhir/project/resources/Specimen-BS-00009-0.json
2019-11-13 17:50:17,299 - kf_model_fhir.loader - DEBUG - Reading resource file: /Users/singhn4/Projects/kids_first/kf-model-fhir/project/resources/Specimen-BS-00009-1.json


*********************
Data Description
*********************


,resource_type,id,references
0,Condition,CD-00000-0,Patient/PT-00000
1,Condition,CD-00001-0,Patient/PT-00001
2,Condition,CD-00002-0,Patient/PT-00002
3,Condition,CD-00003-0,Patient/PT-00003
4,Condition,CD-00004-0,Patient/PT-00004
5,Condition,CD-00005-0,Patient/PT-00005
6,Condition,CD-00006-0,Patient/PT-00006
7,Condition,CD-00007-0,Patient/PT-00007
8,Condition,CD-00008-0,Patient/PT-00008
9,Condition,CD-00009-0,Patient/PT-00009


resource_type
Condition      10
Observation    10
Patient        10
Specimen       20
dtype: int64

## Load Data

In [3]:
# Publish profiles and resources to server
for server_name in ['hapi']:
    # Profiles
    server_settings = SERVER_CONFIG[server_name]
    print(f'\n\n********************** {server_name.upper()} **********************')
    username = server_settings.get('username')
    password = server_settings.get('password')    
    params = ['./profiles', '--resource_type', 'profile',
          '--server_name', server_name]
    if username and password:
        params.extend(['--username', username, '--password', password])    
    
    if server_name == 'aidbox':
        params.extend(['-e', 'search_parameter'])
        load_search_params(username, password)
        
    result = runner.invoke(cli.publish, params)
    assert result.exit_code == 0
    
    # Resources
    params = ['./resources', '--resource_type', 'resource',
          '--server_name', server_name]
    if username and password:
        params.extend(['--username', username, '--password', password])

    result = runner.invoke(cli.publish,params)
    assert result.exit_code == 0    
    print(f'\n\n*******************************************************************')

2019-11-13 17:49:20,608 - kf_model_fhir.app - INFO - Begin publishing profile in ./profiles to Simplifier project http://hapi.fhir.org/baseR4
2019-11-13 17:49:20,772 - FhirValidator - INFO - Starting FHIR 4.0.0 profile validation for ./profiles
2019-11-13 17:49:20,774 - kf_model_fhir.loader - DEBUG - Reading resource file: /Users/singhn4/Projects/kids_first/kf-model-fhir/project/profiles/extensions
2019-11-13 17:49:20,774 - kf_model_fhir.loader - WARNING - Skipping "extensions",  is an invalid resource file type
2019-11-13 17:49:20,775 - kf_model_fhir.loader - DEBUG - Reading resource file: /Users/singhn4/Projects/kids_first/kf-model-fhir/project/profiles/Patient.json
2019-11-13 17:49:20,777 - kf_model_fhir.loader - DEBUG - Reading resource file: /Users/singhn4/Projects/kids_first/kf-model-fhir/project/profiles/search_parameters
2019-11-13 17:49:20,778 - kf_model_fhir.loader - WARNING - Skipping "search_parameters",  is an invalid resource file type
2019-11-13 17:49:20,778 - kf_model_f



********************** HAPI **********************


2019-11-13 17:49:20,876 - FhirApiClient - DEBUG - GET http://hapi.fhir.org/baseR4/StructureDefinition?url=http://fhirr4.kids-first.io/fhir/StructureDefinition/Patient succeeded. Response:
{'id': 'fe2b9eb7-1a1f-40c2-a762-cb10fbfe4ae6',
 'link': [{'relation': 'self',
           'url': 'http://hapi.fhir.org/baseR4/StructureDefinition?url=http%3A%2F%2Ffhirr4.kids-first.io%2Ffhir%2FStructureDefinition%2FPatient'}],
 'meta': {'lastUpdated': '2019-11-13T22:49:21.272+00:00'},
 'resourceType': 'Bundle',
 'total': 0,
 'type': 'searchset'}
2019-11-13 17:49:20,877 - FhirApiClient - DEBUG - Fetched 0 item(s) from http://hapi.fhir.org/baseR4/StructureDefinition?url=http://fhirr4.kids-first.io/fhir/StructureDefinition/Patient
2019-11-13 17:49:20,878 - FhirApiClient - INFO - Begin deleting resources ...
2019-11-13 17:49:20,915 - FhirApiClient - DEBUG - GET http://hapi.fhir.org/baseR4/StructureDefinition?url=http://fhirr4.kids-first.io/fhir/StructureDefinition/us-core-condition succeeded. Response:
{'i

2019-11-13 17:49:21,754 - FhirApiClient - INFO - ✅ POST USCoreEthnicityExtension.json to http://hapi.fhir.org/baseR4/StructureDefinition succeeded
2019-11-13 17:49:21,755 - FhirApiClient - INFO - POSTing FHIR StructureDefinition from USCoreRaceExtension.json
2019-11-13 17:49:21,923 - FhirApiClient - DEBUG - POST http://hapi.fhir.org/baseR4/StructureDefinition succeeded. Response:
{'abstract': False,
 'baseDefinition': 'http://hl7.org/fhir/StructureDefinition/Extension',
 'contact': [{'telecom': [{'system': 'url',
                           'value': 'http://www.healthit.gov'}]}],
 'context': [{'expression': 'Patient', 'type': 'element'}],
 'date': '2019-05-21T00:00:00-04:00',
 'derivation': 'constraint',
 'description': 'Concepts classifying the person into a named category of '
                'humans sharing common history, traits, geographical origin or '
                'nationality.  The race codes used to represent these concepts '
                'are based upon the [CDC Race and

2019-11-13 17:49:21,924 - FhirApiClient - INFO - ✅ POST USCoreRaceExtension.json to http://hapi.fhir.org/baseR4/StructureDefinition succeeded
2019-11-13 17:49:21,925 - FhirValidator - INFO - ✅ Extension validation passed!
2019-11-13 17:49:21,926 - FhirValidator - INFO - Begin reference extension validation ...
2019-11-13 17:49:21,927 - FhirValidator - INFO - Validating reference extensions in Patient.json
2019-11-13 17:49:21,928 - FhirValidator - DEBUG - Checking if http://fhirr4.kids-first.io/fhir/StructureDefinition/us-core-race exists on server
2019-11-13 17:49:22,002 - FhirApiClient - DEBUG - GET http://hapi.fhir.org/baseR4/StructureDefinition?url=http://fhirr4.kids-first.io/fhir/StructureDefinition/us-core-race&_total=accurate succeeded. Response:
{'entry': [{'fullUrl': 'http://hapi.fhir.org/baseR4/StructureDefinition/61906',
            'resource': {'abstract': False,
                         'baseDefinition': 'http://hl7.org/fhir/StructureDefinition/Extension',
                 

...

2019-11-13 17:49:32,853 - FhirApiClient - INFO - ✅ PUT Observation-OB-00005-0.json to http://hapi.fhir.org/baseR4/Observation/OB-00005-0 succeeded
2019-11-13 17:49:32,854 - FhirApiClient - INFO - PUTing FHIR Observation from Observation-OB-00006-0.json
2019-11-13 17:49:33,075 - FhirApiClient - DEBUG - PUT http://hapi.fhir.org/baseR4/Observation/OB-00006-0 succeeded. Response:
{'code': {'coding': [{'code': '15074-8',
                      'display': 'Glucose [Moles/volume] in Blood',
                      'system': 'http://loinc.org'}]},
 'effectivePeriod': {'start': '2013-04-02T09:30:10+01:00'},
 'id': 'OB-00006-0',
 'identifier': [{'value': 'OB-00006-0'}],
 'interpretation': [{'coding': [{'code': 'H',
                                 'display': 'High',
                                 'system': 'http://terminology.hl7.org/CodeSystem/v3-ObservationInterpretation'}]}],
 'issued': '2013-04-03T15:30:10+01:00',
 'meta': {'lastUpdated': '2019-11-13T22:49:33.324+00:00', 'versionId': '1'},
 '

2019-11-13 17:49:33,465 - FhirApiClient - INFO - ✅ PUT Observation-OB-00008-0.json to http://hapi.fhir.org/baseR4/Observation/OB-00008-0 succeeded
2019-11-13 17:49:33,466 - FhirApiClient - INFO - PUTing FHIR Observation from Observation-OB-00009-0.json
2019-11-13 17:49:33,614 - FhirApiClient - DEBUG - PUT http://hapi.fhir.org/baseR4/Observation/OB-00009-0 succeeded. Response:
{'code': {'coding': [{'code': '15074-8',
                      'display': 'Glucose [Moles/volume] in Blood',
                      'system': 'http://loinc.org'}]},
 'effectivePeriod': {'start': '2013-04-02T09:30:10+01:00'},
 'id': 'OB-00009-0',
 'identifier': [{'value': 'OB-00009-0'}],
 'interpretation': [{'coding': [{'code': 'H',
                                 'display': 'High',
                                 'system': 'http://terminology.hl7.org/CodeSystem/v3-ObservationInterpretation'}]}],
 'issued': '2013-04-03T15:30:10+01:00',
 'meta': {'lastUpdated': '2019-11-13T22:49:33.941+00:00', 'versionId': '1'},
 '

2019-11-13 17:49:34,368 - FhirApiClient - INFO - ✅ PUT Condition-CD-00003-0.json to http://hapi.fhir.org/baseR4/Condition/CD-00003-0 succeeded
2019-11-13 17:49:34,369 - FhirApiClient - INFO - PUTing FHIR Condition from Condition-CD-00004-0.json
2019-11-13 17:49:34,552 - FhirApiClient - DEBUG - PUT http://hapi.fhir.org/baseR4/Condition/CD-00004-0 succeeded. Response:
{'abatementDateTime': '2016-01-28T03:52:16-05:00',
 'category': [{'coding': [{'code': 'encounter-diagnosis',
                           'display': 'Encounter Diagnosis',
                           'system': 'http://terminology.hl7.org/CodeSystem/condition-category'}]}],
 'clinicalStatus': {'coding': [{'code': 'resolved',
                                'system': 'http://terminology.hl7.org/CodeSystem/condition-clinical'}]},
 'code': {'coding': [{'code': '87979003',
                      'display': 'Cleft Palate',
                      'system': 'http://snomed.info/sct'}],
          'text': 'Cleft Palate'},
 'id': 'CD-00004-

2019-11-13 17:49:35,603 - FhirApiClient - INFO - ✅ PUT Condition-CD-00009-0.json to http://hapi.fhir.org/baseR4/Condition/CD-00009-0 succeeded
2019-11-13 17:49:35,608 - kf_model_fhir.cli - ERROR - ❌ Publish resource files failed!


AssertionError: 

# Search API Exploration

## Basic Queries

In [6]:
queries = [
    {
        'desc': 'Get all patients',
        'endpoint': 'Patient',
        'params': {}
    },
    {
        'desc': 'Get all female patients',            
        'endpoint': 'Patient',
        'params': {'gender': 'female'}
    },
    {
        'desc': 'Get all female patients with last name = Holmes',        
        'endpoint': 'Patient',
        'params': {'gender': 'female', 'family': 'Holmes'}
    }
]
execute_queries(queries)

****************
Get all patients
****************
AIDBOX
------
Success: GET http://localhost:8081/fhir/Patient
✅ Found 10 Patient matching these params {'_total': 'accurate'}
⚠️ ？ The returned total = actual count of Patient. This may or may not be correct. Some servers ignore search parameters that they do not understand, which results in a get all query. 


VONK
----
Success: GET http://localhost:8080/Patient
⚠️ or ✅ Found 0 Patient matching these params {'_total': 'accurate'}


SMILE-CDR
---------
Success: GET https://try.smilecdr.com:8000/Patient
✅ Found 1007 Patient matching these params {'_total': 'accurate'}


HAPI
----
Success: GET http://hapi.fhir.org/baseR4/Patient
✅ Found 3541 Patient matching these params {'_total': 'accurate'}


AZURE
-----
Success: GET https://kids-first-fhir-service.azurewebsites.net/Patient
✅ Found 10 Patient matching these params {'_total': 'accurate'}
⚠️ ？ The returned total = actual count of Patient. This may or may not be correct. Some servers ign

## Using Modifiers

These are strings that start and/or end with `:` and get appended onto the name of the search parameter
you're using in a query. 

In [7]:
queries = [
    {
        'desc': 'Get all patients missing the gender attribute',
        'endpoint': 'Patient',
        'params': {'gender:missing': True}
    },
    {
        'desc': 'Get all patients that are NOT female',            
        'endpoint': 'Patient',
        'params': {'gender:not': 'female'}
    },
    {
        'desc': 'Get all female patients with name containing Hol',        
        'endpoint': 'Patient',
        'params': {'gender': 'female', 'name:contains': 'Hol'}
    },
]
execute_queries(queries)

*********************************************
Get all patients missing the gender attribute
*********************************************
AIDBOX
------
Success: GET http://localhost:8081/fhir/Patient
✅ Found 10 Patient matching these params {'_total': 'accurate', 'gender:missing': True}
⚠️ ？ The returned total = actual count of Patient. This may or may not be correct. Some servers ignore search parameters that they do not understand, which results in a get all query. 


VONK
----
Success: GET http://localhost:8080/Patient
⚠️ or ✅ Found 0 Patient matching these params {'_total': 'accurate', 'gender:missing': True}


SMILE-CDR
---------
Error with request. Status: 405. Caused by: {'issue': [{'code': 'processing',
            'diagnostics': ':missing modifier is disabled on this server',
            'severity': 'error'}],
 'resourceType': 'OperationOutcome'}
Success: GET https://try.smilecdr.com:8000/Patient
❌ Found None Patient matching these params {'_total': 'accurate', 'gender:missing

## Comparison Operators

These represent comparators like >, <, =, <=, >=, etc.

In [8]:
queries = [
    {
        'desc': 'Get all glucose in blood observations with value > 5 mmol/l',
        'endpoint': 'Observation',
        'params': {'value-quantity': 'gt5', 'code': '15074-8'}
    }
]
execute_queries(queries)

***********************************************************
Get all glucose in blood observations with value > 5 mmol/l
***********************************************************
AIDBOX
------
Success: GET http://localhost:8081/fhir/Observation
✅ Found 1 Observation matching these params {'_total': 'accurate', 'code': '15074-8', 'value-quantity': 'gt5'}


VONK
----
Success: GET http://localhost:8080/Observation
⚠️ or ✅ Found 0 Observation matching these params {'_total': 'accurate', 'code': '15074-8', 'value-quantity': 'gt5'}


SMILE-CDR
---------
Success: GET https://try.smilecdr.com:8000/Observation
✅ Found 4 Observation matching these params {'_total': 'accurate', 'code': '15074-8', 'value-quantity': 'gt5'}


HAPI
----
Success: GET http://hapi.fhir.org/baseR4/Observation
✅ Found 2 Observation matching these params {'_total': 'accurate', 'code': '15074-8', 'value-quantity': 'gt5'}


AZURE
-----
Success: GET https://kids-first-fhir-service.azurewebsites.net/Observation
✅ Found 2 Obse

## Searching Coded Things

Tokens are basically coded things - its an attribute with a code, that comes from a system, and has associated
text too. If a SearchParameter is of type `token` this means when used in a query, by default, the token's
code is searched.

For example Specimen.bodysite is a `token` type search parameter. That means
you can search for specimens by bodysite codes like this: /Specimen?bodysite=49852007
    
Also by default (if server supports it) you can search for a token by its text like this:
/Specimen?bodysite:text=<text version representation of 49852007>

In [9]:
queries = [
    {
        'desc': 'Get all specimens with code text = Left median cubital vein',
        'endpoint': 'Specimen',
        'params': {'bodysite:text': 'Left median cubital vein'}
    },
    {
        'desc': 'Get all specimens with bodysite code = 49852007 (Left median cubital vein)',            
        'endpoint': 'Specimen',
        'params': {'bodysite': '49852007'}
    },
    {
        'desc': 'Get all anemia conditions by code',            
        'endpoint': 'Condition',
        'params': {'code': '271737000'}
    }
]
execute_queries(queries)

***********************************************************
Get all specimens with code text = Left median cubital vein
***********************************************************
AIDBOX
------
Success: GET http://localhost:8081/fhir/Specimen
✅ Found 9 Specimen matching these params {'_total': 'accurate', 'bodysite:text': 'Left median cubital vein'}


VONK
----
Success: GET http://localhost:8080/Specimen
⚠️ or ✅ Found 0 Specimen matching these params {'_total': 'accurate', 'bodysite:text': 'Left median cubital vein'}


SMILE-CDR
---------
Success: GET https://try.smilecdr.com:8000/Specimen
✅ Found 9 Specimen matching these params {'_total': 'accurate', 'bodysite:text': 'Left median cubital vein'}


HAPI
----
Success: GET http://hapi.fhir.org/baseR4/Specimen
✅ Found 9 Specimen matching these params {'_total': 'accurate', 'bodysite:text': 'Left median cubital vein'}


AZURE
-----
Success: GET https://kids-first-fhir-service.azurewebsites.net/Specimen
✅ Found 9 Specimen matching these par

## Search Using References

In [10]:
# Get a sample patient name
from pprint import pprint
c = SERVER_CONFIG['azure']
patient, sc = get(f"{c['base_url']}/Patient/PT-00001")
patient_name = patient['name'][0]['given'][0]

queries = [
    {
        'desc': 'Get all specimens for a patient by patient ID',
        'endpoint': 'Specimen',
        'params': {'subject:Patient': "PT-00001"}
    },
    {
        'desc': 'Get all specimens for a patient using their name (chained search parameters)',
        'endpoint': 'Specimen',
        'params': {'subject:Patient.name': patient_name}
    },
    {
        'desc': 'Get patients with a specimen that has body site denoted by <code> (reverse chained parameters)',
        'endpoint': 'Patient',
        'params': {'_has:Specimen:patient:bodysite': '49852007'}
    },
    {
        'desc': 'Get all the patients that have a Specimen where its Observation.status=final',
        'endpoint': 'Patient',
        'params': {'_has:Specimen:patient:_has:Observation:status': 'final'}
    },
]
execute_queries(queries)

Success: GET https://kids-first-fhir-service.azurewebsites.net/Patient/PT-00001
*********************************************
Get all specimens for a patient by patient ID
*********************************************
AIDBOX
------
Success: GET http://localhost:8081/fhir/Specimen
✅ Found 2 Specimen matching these params {'_total': 'accurate', 'subject:Patient': 'PT-00001'}


VONK
----
Success: GET http://localhost:8080/Specimen
⚠️ or ✅ Found 0 Specimen matching these params {'_total': 'accurate', 'subject:Patient': 'PT-00001'}


SMILE-CDR
---------
Success: GET https://try.smilecdr.com:8000/Specimen
✅ Found 2 Specimen matching these params {'_total': 'accurate', 'subject:Patient': 'PT-00001'}


HAPI
----
Success: GET http://hapi.fhir.org/baseR4/Specimen
✅ Found 2 Specimen matching these params {'_total': 'accurate', 'subject:Patient': 'PT-00001'}


AZURE
-----
Error with request. Status: 403. Caused by: {'id': '6bbde43e-5564-43b1-89be-25bca75a3d6a',
 'issue': [{'code': 'forbidden',
   

## Include Referenced Resources

In [11]:
queries = [
    {
        'desc': 'Get a specimens with its patient',
        'endpoint': 'Specimen',
        'params': {'identifier': "BS-00001-0",
         '_include': 'Specimen:patient'}
    },
    {
        'desc': 'Get an observation and the specimen it is about',
        'endpoint': 'Observation',
        'params': {'identifier': "OB-00001-0",
         '_include': 'Observation:specimen'}
    }
]
execute_queries(queries, display_content=True)


********************************
Get a specimens with its patient
********************************
AIDBOX
------
Success: GET http://localhost:8081/fhir/Specimen
{'entry': [{'fullUrl': 'http://localhost:8081/Patient/PT-00001',
            'resource': {'extension': [{'extension': [{'url': 'text',
                                                       'valueString': 'Filipino'},
                                                      {'url': 'detailed',
                                                       'valueCoding': {'code': '2036-2',
                                                                       'display': 'Filipino',
                                                                       'system': 'urn:oid:2.16.840.1.113883.6.238'}}],
                                        'url': 'http://fhirr4.kids-first.io/fhir/StructureDefinition/us-core-race'},
                                       {'extension': [{'url': 'text',
                                                       'v

Error with request. Status: 403. Caused by: {'id': '89b154f0-6493-4120-a5cd-20c11c0155fd',
 'issue': [{'code': 'forbidden',
            'diagnostics': 'Include expression is not supported.',
            'severity': 'error'}],
 'resourceType': 'OperationOutcome'}
Success: GET https://kids-first-fhir-service.azurewebsites.net/Specimen
{'id': '89b154f0-6493-4120-a5cd-20c11c0155fd',
 'issue': [{'code': 'forbidden',
            'diagnostics': 'Include expression is not supported.',
            'severity': 'error'}],
 'resourceType': 'OperationOutcome'}
❌ Found None Specimen matching these params {'_include': 'Specimen:patient',
 '_total': 'accurate',
 'identifier': 'BS-00001-0'}


***********************************************
Get an observation and the specimen it is about
***********************************************
AIDBOX
------
Success: GET http://localhost:8081/fhir/Observation
{'entry': [{'fullUrl': 'http://localhost:8081/Specimen/BS-00001-1',
            'resource': {'collection'

           {'fullUrl': 'https://try.smilecdr.com:8000/Specimen/BS-00001-1',
            'resource': {'collection': {'bodySite': {'coding': [{'code': '49852007',
                                                                 'display': 'Structure '
                                                                            'of '
                                                                            'median '
                                                                            'cubital '
                                                                            'vein '
                                                                            '(body '
                                                                            'structure)',
                                                                 'system': 'http://snomed.info/sct'}],
                                                     'text': 'Left median '
                                                         

## Full Text Search

In [12]:
queries = [
    {
        'desc': 'Full text search - get all specimens w/ code text = Left median cubital vein',
        'endpoint': 'Specimen',
        'params': {'_content': '"Left median cubital vein"'}
    }
]
execute_queries(queries)

****************************************************************************
Full text search - get all specimens w/ code text = Left median cubital vein
****************************************************************************
AIDBOX
------
Success: GET http://localhost:8081/fhir/Specimen
✅ Found 9 Specimen matching these params {'_content': '"Left median cubital vein"', '_total': 'accurate'}


VONK
----
Success: GET http://localhost:8080/Specimen
⚠️ or ✅ Found 0 Specimen matching these params {'_content': '"Left median cubital vein"', '_total': 'accurate'}


SMILE-CDR
---------
Success: GET https://try.smilecdr.com:8000/Specimen
✅ Found 26 Specimen matching these params {'_content': '"Left median cubital vein"', '_total': 'accurate'}


HAPI
----
Success: GET http://hapi.fhir.org/baseR4/Specimen
✅ Found 209 Specimen matching these params {'_content': '"Left median cubital vein"', '_total': 'accurate'}


AZURE
-----
Success: GET https://kids-first-fhir-service.azurewebsites.net/Spec

## Custom Search Parameter

When you add an extension to a resource, you must create a SearchParameter in order to search for resources
by that extension. For example, you've created an `race` extension and use that on Patient resources. Now
you want to do searches like this: /Patient?race=2028-9 or /Patient?race:text=Asian. In order to do that
you will need to create a SearchParameter for the race extension.

The example queries below demonstrate searches with a custom search parameter

In [13]:
queries = [
    {
        'desc': 'Search on extension - get all patients with particular race',
        'endpoint': 'Patient',
        'params': {'race': '2028-9'}
    }
]
execute_queries(queries)

***********************************************************
Search on extension - get all patients with particular race
***********************************************************
AIDBOX
------
Error with request. Status: 500. Caused by: {'id': 'exception',
 'issue': [{'code': 'exception',
            'diagnostics': 'Search for {:resourceType :Patient, :type :param, '
                           ':name "race", :values [{:value "2028-9"}], '
                           ':search-param #object[clojure.lang.Agent '
                           '0x1caa8855 {:status :ready, :val nil}]} is not '
                           'implemented',
            'severity': 'fatal'}],
 'resourceType': 'OperationOutcome',
 'text': {'div': 'Search for {:resourceType :Patient, :type :param, :name '
                 '"race", :values [{:value "2028-9"}], :search-param '
                 '#object[clojure.lang.Agent 0x1caa8855 {:status :ready, :val '
                 'nil}]} is not implemented',
          'status': '

## Custom Search Query

Sometimes the RESTful FHIR search API cannot satisfy your query needs. It would be nice if you could query the 
database directly. 

In [14]:
# Only Aidbox supports this
from requests.auth import HTTPBasicAuth

c = SERVER_CONFIG['aidbox']
sql_str = (
    """
    SELECT
    p.id AS patient_id,
    s.id AS specimen_id,
    s.resource AS specimen    
    FROM 
    patient AS p
    JOIN specimen AS s
    ON p.id = s.resource->'subject'->>'id';
    """
)
url = f"{c['base_url'].rstrip('/fhir')}/$sql"
resp = requests.post(
    url,
    auth=HTTPBasicAuth(c['username'], c['password']),
    data=sql_str,
    headers={'Content-Type': 'text/yaml'}
)
pprint(resp.json())

[{'patient_id': 'PT-00000',
  'specimen': {'collection': {'bodySite': {'coding': [{'code': '49852008',
                                                       'display': 'Structure '
                                                                  'of median '
                                                                  'cubital '
                                                                  'vein (body '
                                                                  'structure)',
                                                       'system': 'http://snomed.info/sct'}],
                                           'text': 'Right median cubital vein'},
                              'method': {'coding': [{'code': 'LNV',
                                                     'system': 'http://terminology.hl7.org/CodeSystem/v2-0488'}]}},
               'extension': [{'url': 'http://fhirr4.kids-first.io/fhir/StructureDefinition/specimen-analyte-type',
                             

## Exact Path Match

In [15]:
queries = [
    {
        'desc': 'Get all specimens with analyte type = DNA',
        'endpoint': 'Specimen',
        'params': {'.extension.0.valueString': 'DNA'}
    }
]
execute_queries(queries)

*****************************************
Get all specimens with analyte type = DNA
*****************************************
AIDBOX
------
Success: GET http://localhost:8081/fhir/Specimen
✅ Found 12 Specimen matching these params {'.extension.0.valueString': 'DNA', '_total': 'accurate'}


VONK
----
Success: GET http://localhost:8080/Specimen
⚠️ or ✅ Found 0 Specimen matching these params {'.extension.0.valueString': 'DNA', '_total': 'accurate'}


SMILE-CDR
---------
Error with request. Status: 400. Caused by: {'issue': [{'code': 'processing',
            'diagnostics': 'Unknown search parameter "". Value search '
                           'parameters for this search are: [_id, _language, '
                           'accession, bodysite, collected, collector, '
                           'container, container-id, identifier, parent, '
                           'patient, status, subject, type]',
            'severity': 'error'}],
 'resourceType': 'OperationOutcome'}
Success: GET http